In [1]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range

In [2]:
import nltk
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from sklearn.linear_model import LogisticRegression

In [3]:
from collections import Counter
from bs4 import BeautifulSoup

In [4]:
pd.set_option('display.max_colwidth', None)

source: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [5]:
tknzr = TweetTokenizer()

In [6]:
wordnet_lemmatizer = WordNetLemmatizer()

In [7]:
#load
df = pd.read_csv('Mar21-vaccine-uptake-main 3/data/twitter/twitter_data_cleaned.csv')
df

tweet_id                 created_at  \
0      1.364223e+18  2021-02-23 14:38:16+00:00   
1      1.364381e+18  2021-02-24 01:07:52+00:00   
2      1.364610e+18  2021-02-24 16:14:15+00:00   
3      1.364727e+18  2021-02-24 23:59:58+00:00   
4      1.364727e+18  2021-02-24 23:59:58+00:00   
...             ...                        ...   
51152  1.366553e+18  2021-03-02 00:55:24+00:00   
51153  1.366549e+18  2021-03-02 00:42:14+00:00   
51154  1.366547e+18  2021-03-02 00:31:41+00:00   
51155  1.366544e+18  2021-03-02 00:20:03+00:00   
51156  1.366543e+18  2021-03-02 00:16:03+00:00   

                                                                                                                                                                                                                                                                                                full_text  \
0      Here’s what's in the COVID relief package:\n \nDirect checks to families\nUnemployment assistance\nMoney to reopen schools\nVaccine funding\nFood aid\n$15 minimum wage\nChild care funding\nRental assistance\nEviction moratorium\n \nAll popular. All necessary. All urgent. Let’s get it done.   
1                                                                                                   Will the National Endowment for the Arts be helping with vaccine distribution?\n\nWill they be producing masks suddenly?\n\nIf not, why are they being given $135 million in the “COVID Relief Bill”?   
2                                                                                                                                   This is both anecdotal and early, but many long covid survivors are feeling significantly better after receiving their first vaccine dose. Including me. Fascinating.   
3                                                                    A Link to Professor Chossudovsky’s Analysis of What #Covid Is Really About\nhttps://t.co/glJ9vnnm3o \n\n#covidHOAX  #PLANdemic  #Event201  #Scamdemic  #ChinaVirus  #MedicalMartialLaw  #CCPvirus  #coronavirus  #covid19  #covid_19   
4                 Children warned over hugging grandparents even if they've had vaccine https://t.co/qd5pxDHlFN A small story, but it sums up the Covid debate. There is nothing resembling evidence for this prohibit, and a huge cost to two politically weak groups - the very young and the very old.   
...                                                                                                                                                                                                                                                                                                   ...   
51152                 EU - GERMANY ....and the beat goes on.... Where are the places in Germany to be Vaccinated? Do Germans have choice which Vaccine to select? Will Germans see a "slack" from CORONA(MANIA)DICTATORSHIP next Wednesday's after Merkel's "meeting"...? Or a mere "talk-Show" again? 😏😠   
51153                                                                  @satyendrajain @ArvindKejriwal @DrKKAggarwal  Vaccination of frontline officers is being delayed in Power Department -Power Plants.Very few have been vaccinated while new Corona cases in power plants #Delhi Government dullness   
51154                                                                                                                                                                           Do it the Coward and Liar way. Get vaccinated and tell no one. #trump got vaccinated in January!  https://t.co/E5XLYPJ4UN   
51155                                                                                                                                                                                                               Trump urges supporters to get vaccinated against corona virus https://t.co/Ytq6iAbHIH   
51156                                                                                              

In [8]:
df['copy_index'] = df.index
#df

In [27]:
#loads
stopwords = set(w.rstrip() for w in open('stopwords/stopwords.txt'))

In [10]:
#df.info()

In [11]:
#text
positive = set(w.rstrip() for w in open('positive_emotions/positive-words.txt'))
negative = set(w.rstrip() for w in open('afinn-master/afinn/data/AFINN-en-165.txt'))

# Positive- Negative

In [12]:
from textblob import TextBlob

In [13]:
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

In [14]:
df['polarity'] = df['full_text'].apply(pol)
df['subjectivity'] = df['full_text'].apply(sub)

# Natural Language Processing 

source: https://www.dataquest.io/blog/natural-language-processing-with-python/

In [24]:
def tokenizer(s):
    s = s.lower() # downcase
    tokens = tknzr.tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens 

In [25]:
new_dataframe = []
for index, i in enumerate(zip(df['copy_index'], df['full_text'])):
    copy_index= i[0]
    text = tokenizer(i[1])
    
    new_dataframe.append({'copy_index' : copy_index,
                       'text': text})

In [26]:
new_df = pd.DataFrame(new_dataframe, columns = ['copy_index', 'text'])
new_df

copy_index  \
0               0   
1               1   
2               2   
3               3   
4               4   
...           ...   
51152       51152   
51153       51153   
51154       51154   
51155       51155   
51156       51156   

                                                                                                                                                                                                                                        text  
0      [what's, covid, relief, package, direct, check, family, unemployment, assistance, money, reopen, school, vaccine, funding, food, aid, minimum, wage, child, care, funding, rental, assistance, eviction, moratorium, popular, urgent]  
1                                                                                                                   [national, endowment, art, helping, vaccine, distribution, producing, mask, suddenly, 135, million, covid, relief, bill]  
2                                                                                                                                     [anecdotal, covid, survivor, feeling, significantly, receiving, vaccine, dose, including, fascinating]  
3                                   [link, professor, chossudovsky, analysis, #covid, https://t.co/glj9vnnm3o, #covidhoax, #plandemic, #event201, #scamdemic, #chinavirus, #medicalmartiallaw, #ccpvirus, #coronavirus, #covid19, #covid_19]  
4                                                                 [child, warned, hugging, grandparent, they've, vaccine, https://t.co/qd5pxdhlfn, story, sum, covid, debate, resembling, evidence, prohibit, huge, cost, politically, weak]  
...                                                                                                                                                                                                                                      ...  
51152                                                      [germany, ..., beat, ..., germany, vaccinated, german, choice, vaccine, select, german, slack, corona, mania, dictatorship, wednesday's, merkel's, meeting, ..., mere, talk-show]  
51153                                      [@satyendrajain, @arvindkejriwal, @drkkaggarwal, vaccination, frontline, officer, delayed, power, department, power, plants.very, vaccinated, corona, power, plant, #delhi, government, dullness]  
51154                                                                                                                                                 [coward, liar, vaccinated, tell, #trump, vaccinated, january, https://t.co/e5xlypj4un]  
51155                                                                                                                                                           [trump, urge, supporter, vaccinated, corona, virus, https://t.co/ytq6iabhih]  
51156                                                                             [courtesy, @voxdotcom, realize, efficacy, doesn't, matter, zero, people, died, #corona, vaccinated, hence, 100, reduction, death, https://t.co/iifa0ia0um]  

[51157 rows x 2 columns]

In [19]:
#testing
#text = new_df['text']

# Make the Matrix 

In [20]:
# now let's create our input matrices
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1) # last element is for the label
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum() # normalize it before setting label
    x[-1] = label
    return x


In [21]:
N = len(positive_tokenized) + len(negative_tokenized)
# (N x D+1 matrix - keeping them together for now so we can shuffle more easily later
data = np.zeros((N, len(word_index_map) + 1))
i = 0
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1

NameError: name 'positive_tokenized' is not defined

In [ ]:
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [ ]:
# shuffle the data and create train/test splits
# try it multiple times!
this_texts, data = shuffle(this_texts, data)

X = data[:,:-1]
Y = data[:,-1]

In [ ]:
# last 100 rows will be test
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

In [ ]:
#model = LogisticRegression()
#model.fit(Xtrain, Ytrain)
#print("Train accuracy:", model.score(Xtrain, Ytrain))
#print("Test accuracy:", model.score(Xtest, Ytest))

In [ ]:
#model.fit(Xtrain, Ytrain)
